#Kaggle에서 실습할 경우
① Add data 클릭 → ② "metal surface defects" 검색 → ③ Add 클릭

이후 우측 상단 input 폴더를 클릭하여  "neu-metal-surface-defects-data" 가 추가된 것을 확인

#사내 PC에서 실습할 경우
아래 링크로 데이터를 다운로드 받은 후, D드라이브 작업방에 압축을 해제해 놓으세요.

데이터셋 : https://www.kaggle.com/datasets/fantacher/neu-metal-surface-defects-data/download

### 데이터 로드

In [6]:
#모듈 불러오기
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [7]:
#root dir 설정
# root_dir = "../input/neu-metal-surface-defects-data/NEU Metal Surface Defects Data" #캐글에서 실습할 경우
root_dir = "D:/작업방/NEU Metal Surface Defects Data" #PC에서 실습할 경우

#os.listdir() 코드로 폴더 탐색
os.listdir(root_dir + "/train")

['Crazing', 'Inclusion', 'Patches', 'Pitted', 'Rolled', 'Scratches']

In [8]:
#os.listdir() 코드로 폴더 탐색
os.listdir(root_dir + "/train/Inclusion")

['In_120.bmp',
 'In_121.bmp',
 'In_122.bmp',
 'In_123.bmp',
 'In_124.bmp',
 'In_125.bmp',
 'In_126.bmp',
 'In_127.bmp',
 'In_128.bmp',
 'In_129.bmp',
 'In_13.bmp',
 'In_130.bmp',
 'In_131.bmp',
 'In_132.bmp',
 'In_133.bmp',
 'In_134.bmp',
 'In_135.bmp',
 'In_136.bmp',
 'In_137.bmp',
 'In_138.bmp',
 'In_139.bmp',
 'In_14.bmp',
 'In_140.bmp',
 'In_141.bmp',
 'In_142.bmp',
 'In_143.bmp',
 'In_144.bmp',
 'In_145.bmp',
 'In_146.bmp',
 'In_147.bmp',
 'In_148.bmp',
 'In_149.bmp',
 'In_15.bmp',
 'In_150.bmp',
 'In_151.bmp',
 'In_152.bmp',
 'In_153.bmp',
 'In_154.bmp',
 'In_155.bmp',
 'In_156.bmp',
 'In_157.bmp',
 'In_158.bmp',
 'In_159.bmp',
 'In_16.bmp',
 'In_160.bmp',
 'In_161.bmp',
 'In_162.bmp',
 'In_163.bmp',
 'In_164.bmp',
 'In_165.bmp',
 'In_166.bmp',
 'In_167.bmp',
 'In_168.bmp',
 'In_169.bmp',
 'In_17.bmp',
 'In_170.bmp',
 'In_171.bmp',
 'In_172.bmp',
 'In_173.bmp',
 'In_174.bmp',
 'In_175.bmp',
 'In_176.bmp',
 'In_177.bmp',
 'In_178.bmp',
 'In_179.bmp',
 'In_18.bmp',
 'In_180.bmp',
 

In [9]:
#폴더명 지정하여 변수로 저장 
train_dir = root_dir + "/train"
val_dir = root_dir + "/valid"
test_dir = root_dir + "/test"

#라벨명을 리스트로 미리 만들어 놓기
labels = os.listdir(train_dir)
labels

['Crazing', 'Inclusion', 'Patches', 'Pitted', 'Rolled', 'Scratches']

In [10]:
#train 하위 폴더별 이미지 개수 파악
for label in labels:
    print("{} : {}개".format(label, len(os.listdir(train_dir+"/"+label))))

Crazing : 276개
Inclusion : 276개
Patches : 276개
Pitted : 276개
Rolled : 276개
Scratches : 276개


### 이미지 확인

In [ ]:
#이미지 하나 불러와서 출력하기
img_file_list = os.listdir(train_dir+"/"+labels[0]+"/")
img_file = train_dir+"/"+labels[0]+"/"+img_file_list[0]
image_pil = Image.open(img_file)
image_pil

In [ ]:
#이미지를 numpy ndarray로 변환
image = np.array(image_pil)
print(image.shape)  #이미지 모양 200 X 200
image

In [ ]:
#numpy ndarray로 변환된 이미지로 pyplot으로 출력
plt.imshow(image, "gray")
plt.show()

### 이미지를 ndarray 묶음으로 변환

In [ ]:
#입력 데이터를 ndarray로 변환하는 사용자 함수 선언
def makeTrainData(image_dir):
    x_list = None #이미지 데이터를 담을 x_list 변수 선언
    y_list = None #라벨 데이터를 담을 y_list 변수 선언
    cnt = 0
    for label_no, label_dir in enumerate(os.listdir(image_dir)):
        for file in os.listdir(image_dir + "/" + label_dir):
            cnt += 1
            print("처리중...({})".format(cnt), end="\r")
            #1.이미지 데이터 입력

            #이미지를 하나씩 불러와 ndarray로 변환한 후 this_img 변수에 저장
            #흑백 이미지이나 컬러로 불러와 3채널로 구성할 경우 모델 성능이 더 잘나옴
            this_img = np.array(Image.open(image_dir + "/" + label_dir + "/" + file).convert("RGB"))

            #이미지를 리스트로 묶어야 하므로 앞 부분에 축을 하나 추가해 줌
            this_img = np.expand_dims(this_img, axis=0) #this_img.shape → (1, 200, 200, 1)

            if x_list is None:
                x_list = this_img
            else:
                x_list = np.vstack([x_list, this_img])


            #2. 라벨 데이터 입력
            this_label = np.zeros(shape=(1, 6)) # → array([[0., 0., 0., 0., 0., 0.]])
            this_label[0, label_no] = 1 # → label_no가 3이라면 array([[0., 0., 0., 1., 0., 0.]]) 로 변경됨

            if y_list is None:
                y_list = this_label
            else:
                y_list = np.vstack([y_list, this_label])
    
    x_list = x_list/255. #0~1사이 값으로 리스케일링
    return x_list, y_list

In [ ]:
#트레이닝 데이터셋 생성
x_train, y_train = makeTrainData(train_dir)
x_train.shape, y_train.shape # → ((1656, 200, 200, 1), (1656, 6))

In [ ]:
#검증 데이터셋 생성
x_val, y_val = makeTrainData(val_dir)
x_val.shape, y_val.shape # → ((72, 200, 200, 1), (72, 6))

### 모델 생성

In [ ]:
#모델 생성
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(input_shape=(200, 200, 3), kernel_size=(3, 3), filters=32, padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(kernel_size=(3, 3), filters=64, padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(kernel_size=(3, 3), filters=128, padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=512, activation="relu"),
    tf.keras.layers.Dense(units=256, activation="relu"),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(units=6, activation="softmax")
])

model.summary() #모델 확인

#모델 컴파일
model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss="categorical_crossentropy", 
              metrics=["accuracy"]
)

#모델 학습
history = model.fit(x_train, y_train, epochs=25, batch_size=32, validation_data=[x_val, y_val])

In [ ]:
#트레이닝 결과 확인
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history["loss"], "b-", label="loss")
plt.plot(history.history["val_loss"], "r--", label="val_loss")
plt.xlabel("Epoch")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history["accuracy"], "g-", label="accuracy")
plt.plot(history.history["val_accuracy"], "k--", label="val_accuracy")
plt.xlabel("Epoch")
plt.ylim(0.7, 1)
plt.legend()

plt.show()

In [ ]:
#케라스 모델로 저장
save_path = "my_model.h5"
model.save(save_path, include_optimizer=True)

### 모델 활용

Kaggle에서 모델을 만든 경우, 아래 절차대로 모델을 다운로드 받은 후 코드를 실행해 주세요.


1. 우측 메뉴에 있는 "/kaggle/working" 폴더를 클릭

2. my_model.h5 우측으로 마우스 이동 후 점세개 모양 버튼 클릭 후 다운로드 클릭

3. pc의 "d:/작업방" 폴더에 "ai" 폴더 생성 (다른 폴더도 상관없음)

4. 캐글에서 다운로드 받은 "my_model.h5" 파일을 "ai" 폴더에 복사

5. pc의 "ai" 폴더 하위에 test_images 폴더 생성 

6. github test_images 폴더에 있는 이미지 파일 16개를 "ai/test_images" 폴더에 복사

7. "ai" 폴더에서 jupyter notebook 실행 

In [ ]:
#모델 불러오기
model = tf.keras.models.load_model("my_model.h5")
model.summary()

In [ ]:
#test 이미지 1개 확인
img_file_list = os.listdir(test_dir+"/"+labels[0]+"/")
img_file = test_dir+"/"+labels[0]+"/"+img_file_list[0]
image_pil = Image.open(img_file).convert("RGB") #컬러 이미지로 불러오기

img = np.array(image_pil) #array로 변환
img = img/255. #0~1 사이로 리스케일링
img = img[tf.newaxis, ...] #앞쪽에 배열 추가,  1,200,200,3 
pred = model.predict(img) #모델을 활용하여 img 예측, pred는 6개의 확률값
classes = np.argmax(pred) #첫번째 배열에서 가장 큰 값의 위치 찾기
confidence = round(np.max(pred)*100, 2) #확률값 계산
names = labels[classes] #class_names 찾기
title = "{} : {}".format(names, confidence) #타이틀 텍스트 만들기	

plt.title(title)
plt.imshow(np.squeeze(img), "gray") #200,200 이미지 출력
plt.show()

In [11]:
#테스트할 이미지 파일을 뽑아 리스트 변수에 추가
test_file_list = []
for dir in os.listdir(test_dir): #test 폴더 하위에 있는 6개 폴더를 반복문으로 접근
    for idx, file in enumerate(os.listdir(test_dir+"/"+dir)): #6개 폴더별 파일들에 반복문으로 접근
        if idx < 4: #클래스별 4개 파일만 뽑아내 test_file_list 변수에 추가
            test_file_list.append(test_dir+"/"+dir+"/"+file)
test_file_list #클래스별 4개 * 6개 클래스 = 총 24개의 이미지 경로가 저장됨

['D:/작업방/NEU Metal Surface Defects Data/test/Crazing/Cr_1.bmp',
 'D:/작업방/NEU Metal Surface Defects Data/test/Crazing/Cr_10.bmp',
 'D:/작업방/NEU Metal Surface Defects Data/test/Crazing/Cr_100.bmp',
 'D:/작업방/NEU Metal Surface Defects Data/test/Crazing/Cr_101.bmp',
 'D:/작업방/NEU Metal Surface Defects Data/test/Inclusion/In_1.bmp',
 'D:/작업방/NEU Metal Surface Defects Data/test/Inclusion/In_10.bmp',
 'D:/작업방/NEU Metal Surface Defects Data/test/Inclusion/In_100.bmp',
 'D:/작업방/NEU Metal Surface Defects Data/test/Inclusion/In_101.bmp',
 'D:/작업방/NEU Metal Surface Defects Data/test/Patches/Pa_1.bmp',
 'D:/작업방/NEU Metal Surface Defects Data/test/Patches/Pa_10.bmp',
 'D:/작업방/NEU Metal Surface Defects Data/test/Patches/Pa_100.bmp',
 'D:/작업방/NEU Metal Surface Defects Data/test/Patches/Pa_101.bmp',
 'D:/작업방/NEU Metal Surface Defects Data/test/Pitted/PS_1.bmp',
 'D:/작업방/NEU Metal Surface Defects Data/test/Pitted/PS_10.bmp',
 'D:/작업방/NEU Metal Surface Defects Data/test/Pitted/PS_100.bmp',
 'D:/작업방/NEU Meta

In [ ]:
#test 이미지 24건을 분류, 확률값이 80% 이하일 경우 적색 텍스트로 출력 
i=1
plt.figure(figsize=(16, 16))
for file in test_file_list:
    image_pil = Image.open(file).convert("RGB")
    img = np.array(image_pil)
    img = img/255.
    img = img[tf.newaxis, ...]
    pred = model.predict(img)
    classes = np.argmax(pred)
    confidence = round(np.max(pred)*100, 2)
    names = labels[classes]
    title = "{} : {}% (True : {})".format(names, confidence, file.split("/")[-1].split("_")[0])
    plt.subplot(6, 4, i)
    if confidence < 80:
        plt.title(title, color="red")
    else:
        plt.title(title, color="blue")
    plt.axis("off")
    plt.imshow(np.squeeze(img), "gray")
    i += 1
plt.show()